In [1]:
import pandas as pd

In [2]:
# FUNCTION DEFINITION: function to create DataFrameMatrix with define row_name and col_name 
def matrix_maker_function(dataframe,row_name,col_name,value_name):
    df = dataframe.filter([row_name,col_name,value_name], axis=1)
    matrix_df = pd.pivot_table(df,index=[row_name],values=[value_name],columns=[col_name], aggfunc='sum')             
    matrix_df = pd.DataFrame(matrix_df.to_records())                                
    matrix_df.columns = [hdr.replace("('", "").replace("', ", "").replace(value_name, "").replace(")", "") \
                     for hdr in matrix_df.columns]
    return matrix_df

In [3]:
def filled_matrix_function(dataframe,row_name,col_name,value_name):
    df = dataframe.filter([row_name,col_name,value_name], axis=1)
    matrix_df = pd.pivot_table(df,index=[row_name],values=[value_name],columns=[col_name], aggfunc='sum')             
    matrix_df = pd.DataFrame(matrix_df.to_records())                                
    matrix_df.columns = [hdr.replace("('", "").replace("', ", "").replace(value_name, "").replace(")", "") \
                     for hdr in matrix_df.columns]
    matrix_df = matrix_df.set_index(row_name).transpose().round(2)
    matrix_df.fillna(method='bfill', inplace=True)
    matrix_df.fillna(method='ffill', inplace=True)
    matrix_df = matrix_df.transpose().reset_index()
    matrix_df = matrix_df[matrix_df[row_name].isin(HappyCountries[row_name])].set_index(row_name)
    matrix_df.drop(matrix_df.columns[[0, 1, 13, 14]], axis = 1, inplace = True)
    return matrix_df

In [4]:
pd.set_option("display.max_rows",None)

The process below are to check Happiness Index data and decide which countries are included.

In [5]:
#Read main Happiness Index file - WHR20_DataForTable2.1.xls
happyindex_file = pd.read_excel("../Resources/WHR20_DataForTable2.1.xls")     
happyindex_file.replace(['Taiwan Province of China'], ['Taiwan'],inplace=True)
country_code = pd.read_csv("../Resources/countrycode1.csv")

In [6]:
country_code.head(1)

Country name Code
0  Afghanistan  AFG

In [7]:
# create matrix for all happiness index
Happy_Index_df = matrix_maker_function(happyindex_file ,'Country name','year','Life Ladder')
print(Happy_Index_df.shape)
Happy_Index_df.head(1)

(166, 16)


Country name  2005  2006  2007     2008      2009      2010      2011  \
0  Afghanistan   NaN   NaN   NaN  3.72359  4.401778  4.758381  3.831719   

       2012    2013      2014      2015      2016      2017      2018  \
0  3.782938  3.5721  3.130896  3.982855  4.220169  2.661718  2.694303   

       2019  
0  2.375092

In [8]:
# check the completeness of data
Happy_Index_df.isna().sum().head(3)

Country name      0
2005            139
2006             77
dtype: int64

HappinessIndex data is the most incomplete for 2005 - and 

In [9]:
THappy_Index_df = Happy_Index_df.set_index('Country name').transpose()
THappy_Index_df.isna().sum().head(1)

Country name
Afghanistan    3
dtype: int64

2005 and 2006 has fairly incomplete data set - thus we eliminate those two years. 
Most country have complete data, but a few has less than 10 data points, we eliminate those countries and come up with data set of 128 countries. 

In [10]:
#create the matrix of countries that we are going to study
df1 = Happy_Index_df.drop(Happy_Index_df.columns[1:3], axis=1).round(2)
df1 = df1.dropna(thresh=10).reset_index(drop=True)
print(df1.shape)
df1.to_csv("../Outputs/HappyIndex_multiyear_mx.csv", index=False)
df1.head(1)

(128, 14)


Country name  2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  \
0  Afghanistan   NaN  3.72   4.4  4.76  3.83  3.78  3.57  3.13  3.98  4.22   

   2017  2018  2019  
0  2.66  2.69  2.38

In [11]:
#check list of rows to be deleted
outcast_df = Happy_Index_df[~Happy_Index_df['Country name'].isin(df1['Country name'])].reset_index(drop=True)
outcast_df.head(1)

Country name  2005  2006  2007  2008  2009      2010      2011      2012  \
0      Algeria   NaN   NaN   NaN   NaN   NaN  5.463567  5.317194  5.604596   

   2013      2014  2015      2016      2017      2018      2019  
0   NaN  6.354898   NaN  5.340854  5.248912  5.043086  4.744627

In [12]:
# create a base data frame of countries that in our country list and the country codes .reset_index(drop=True)
HappyCountries = df1.filter(['Country name'], axis=1)

mapping = dict(country_code[['Country name', 'Code']].values)
HappyCountries['Code'] = country_code['Country name'].map(mapping)
#HappyCountries.to_csv("../Outputs/HappyCountries.csv", index=False)
print(HappyCountries.shape)
HappyCountries.head(1)

(128, 2)


Country name Code
0  Afghanistan  AFG

In [13]:
# drop rows and columns that has too much NaN from original file
new_df = happyindex_file[~happyindex_file['Country name'].isin(outcast_df['Country name'])].round(2)
new_df = new_df[new_df['year'] != (2005,2006)]
new_df = new_df.drop(new_df.columns[9:26], axis=1).reset_index(drop=True)

In [14]:
HI_df = matrix_maker_function(happyindex_file ,'Country name','year','Life Ladder').set_index("Country name").transpose().round(2)
HI_df.fillna(method='bfill', inplace=True)
HI_df.fillna(method='ffill', inplace=True)
HI_df = HI_df.transpose().reset_index()
HI_df = HI_df[HI_df['Country name'].isin(HappyCountries['Country name'])].set_index("Country name")
HI_df.drop(HI_df.columns[[0, 1, 13, 14]], axis = 1, inplace = True)
HI_df.head(1)

2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  2017
Country name                                                                  
Afghanistan   3.72  3.72   4.4  4.76  3.83  3.78  3.57  3.13  3.98  4.22  2.66

In [15]:
#create a list of indicators
indicators = new_df.keys()[2:10]
names = ['Happy_Index_mx','Log_GDP_mx','Soc_support_mx','Life_exp_mx','Freedom_mx','Generosity_mx','Corruption_mx']

print(indicators)
print(names)

Index(['Life Ladder', 'Log GDP per capita', 'Social support',
       'Healthy life expectancy at birth', 'Freedom to make life choices',
       'Generosity', 'Perceptions of corruption'],
      dtype='object')
['Happy_Index_mx', 'Log_GDP_mx', 'Soc_support_mx', 'Life_exp_mx', 'Freedom_mx', 'Generosity_mx', 'Corruption_mx']


In [16]:
#create a list of indicators
indicators = new_df.keys()[2:10]

#for all indicators create matrixes of country v/s year
for i in range (0,len(indicators)):
    df = filled_matrix_function(new_df,'Country name','year',indicators[i])
    df.to_csv(f"../Outputs/New_HI{i}_mx.csv", index=False)

# 2017

Due to the availability to other ....... data, the group decided to evaluate the 2017 data
The following process is creating the a data frame of the happiness index and its 6 factors

In [17]:
H1 = HappyCountries.copy()
df_2017 = new_df[new_df["year"]==2017]
df_2017 = pd.merge(H1, df_2017, how='left', on=('Country name')).reset_index(drop=True)
df_2017.rename(columns={"Life Ladder":"Happiness Index","Log GDP per capita":"Log GDP","Healthy life expectancy at birth":"Healthy life exp",
             "Freedom to make life choices":"Freedom","Perceptions of corruption":"Pct corruption"},inplace=True)

print(df_2017.shape) 
df_2017.head(1)            

(128, 10)


Country name Code    year  Happiness Index  Log GDP  Social support  \
0  Afghanistan  AFG  2017.0             2.66     7.47            0.49   

   Healthy life exp  Freedom  Generosity  Pct corruption  
0              52.8     0.43       -0.11            0.95

In [18]:
GDP1_pp = pd.read_csv("../Resources/gdp_df.csv")
GDP1_pp.head(1)

Country Name  1990  1991  1992  1993  1994  1995  1996  1997  1998  ...  \
0  Afghanistan   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

     2008    2009    2010    2011    2012    2013    2014    2015    2016  \
0  1588.0  1882.0  2094.0  2038.0  2220.0  2264.0  2249.0  2213.0  2201.0   

     2017  
0  2203.0  

[1 rows x 29 columns]

# GHDx Data

In [19]:
#Read main file - IHME-GBD_2017_DATA-ba7d53b0-1 (without age group)
GHDx_file = pd.read_csv("../Resources/IHME-GBD_2017_DATA-ba7d53b0-1.csv")            
GHDx_file.replace(['Taiwan (Province of China)'], ['Taiwan'],inplace=True)
GHDx_map = pd.read_csv("../Resources/GHDx_map.csv")
GHDx_map.head(15)

location_name         Country name Code  Unnamed: 3 Unnamed: 4  \
0           Afghanistan          Afghanistan  AFG         NaN        NaN   
1               Albania              Albania  ALB         NaN        NaN   
2               Algeria              Algeria  DZA         NaN        NaN   
3        American Samoa       American Samoa  NaN         NaN        NaN   
4               Andorra              Andorra  AND         NaN        NaN   
5                Angola               Angola  AGO         NaN        NaN   
6   Antigua and Barbuda  Antigua and Barbuda  ATG         NaN        NaN   
7             Argentina            Argentina  ARG         NaN        NaN   
8               Armenia              Armenia  ARM         NaN        NaN   
9             Australia            Australia  AUS         NaN        NaN   
10              Austria              Austria  AUT         NaN        NaN   
11           Azerbaijan           Azerbaijan  AZE         NaN        NaN   
12              Bahrain              Bahrain  BHR         NaN        NaN   
13           Bangladesh           Bangladesh  BGD         NaN        NaN   
14             Barbados             Barbados  BRB         NaN        NaN   

   Unnamed: 5  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  
6         NaN  
7         NaN  
8         NaN  
9         NaN  
10        NaN  
11        NaN  
12        NaN  
13        NaN  
14        NaN

In [20]:
# DEATH BY SUICIDE RATE/100,000 pop MATRIX
# measure_id = 1 Death, sex_id = 3 All, CauseNo = 718 (selfharm/suicide), MetricId = 3 Rate , year = 2017
suicide_df = GHDx_file[(GHDx_file["measure_id"]==1) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 718) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

# DEPRESSION AND ANXIETY (by Incidents) RATE/100,000 pop MATRIX 
# measure_id = 6 Incidence, sex_id = 3 All, CauseNo = 571 Anxiety disorders, year = 2017
anx_df = GHDx_file[(GHDx_file["measure_id"]==6) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 571) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

# measure_id = 6 Incidence, sex_id = 3 All, CauseNo = 567 Depressive disorders, year = 2017
dep_df = GHDx_file[(GHDx_file["measure_id"]==6) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 567) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

# SUBSTANCE ABUSE (by Incidents) RATE/100,000 pop MATRIX 
# measure_id = 6 Incidence, sex_id = 3 All, CauseNo = 973 Substance Abuse, year = 2017
subst_df = GHDx_file[(GHDx_file["measure_id"]==6) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 973) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

GHDx_df = suicide_df.filter(['location_name','val'], axis=1)
GHDx_df['Depression & Anxiety'] = anx_df["val"] + dep_df["val"] 
GHDx_df['Substance Abuse'] = subst_df["val"] 
GHDx_df = GHDx_df.sort_values(by="location_name").rename(columns={"val":"Suicide"}).reset_index(drop=True)

#GHDx_df.to_csv("../Outputs/GHDx_df.csv", index=False)
print(GHDx_df.shape)
GHDx_df.head(10)

(195, 4)


location_name  Suicide  Depression & Anxiety  Substance Abuse
0          Afghanistan     5.25               3870.85           687.47
1              Albania     5.86               3179.85           642.91
2              Algeria     4.20               4155.35           560.90
3       American Samoa     6.26               3871.54           577.75
4              Andorra    10.24               7021.78           946.03
5               Angola     6.19               3646.34           478.77
6  Antigua and Barbuda     2.95               6796.09          1711.72
7            Argentina    11.35               3666.08          1089.71
8              Armenia     9.97               4525.35           736.38
9            Australia    12.76               3291.17           455.44

In [21]:
mapping1 = dict(GHDx_map[['location_name', 'Country name']].values)
GHDx_df.insert(1, 'Country name', GHDx_map['location_name'].map(mapping1))
GHDx_df.drop(GHDx_df.columns[0], axis=1, inplace=True)
GHDx_df.shape

(195, 4)

In [22]:
GHDx_2017_df = GHDx_df[GHDx_df['Country name'].isin(df_2017['Country name'])].reset_index(drop=True)
print(GHDx_2017_df.shape)
GHDx_2017_df

(127, 4)


Country name  Suicide  Depression & Anxiety  Substance Abuse
0                Afghanistan     5.25               3870.85           687.47
1                    Albania     5.86               3179.85           642.91
2                  Argentina    11.35               3666.08          1089.71
3                    Armenia     9.97               4525.35           736.38
4                  Australia    12.76               3291.17           455.44
5                    Austria    16.17               3709.29          2282.46
6                 Azerbaijan     4.22               3436.33          1070.28
7                    Bahrain     4.96               4959.68          1022.44
8                 Bangladesh     5.95               3219.97           314.86
9                    Belarus    24.83               3088.53           619.74
10                   Belgium    18.83               3702.73          1165.80
11                     Benin     5.99               5171.53           523.21
12                   Bolivia     5.79               4635.47           560.01
13    Bosnia and Herzegovina     9.17               3394.43           858.44
14                  Botswana     8.77               4906.40          1077.76
15                    Brazil     6.68               2975.48           664.17
16                  Bulgaria    14.11               5506.84          1150.66
17              Burkina Faso     7.14               3525.55          2248.75
18                  Cambodia     4.10               3689.59          1397.34
19                  Cameroon     7.82               3642.01           496.19
20                    Canada    12.83               3436.98          1344.46
21                      Chad     5.20               3983.12           895.83
22                     Chile    11.50               2909.21           561.61
23                     China     9.10               3334.08           489.46
24                  Colombia     5.76               3927.70           730.62
25                Costa Rica     9.42               2358.57           707.43
26                   Croatia     8.50               3699.59           575.65
27                    Cyprus    16.55               3802.81           633.27
28                    Cyprus     5.58               4110.12           514.98
29            Czech Republic    14.32               2933.01          1032.09
30                   Denmark    12.12               3517.08           448.12
31        Dominican Republic     8.86               2646.42           833.25
32                   Ecuador     9.36               4583.04           872.79
33                     Egypt     4.93               4456.14           617.72
34               El Salvador    10.07               3911.63           450.98
35                    Guinea     4.92               4127.00           555.51
36                   Estonia    16.70               3699.90           557.77
37                   Finland    15.73               4429.15           527.51
38                    France    16.84               3015.98           531.88
39                     Gabon     9.29               2444.47           530.06
40                   Georgia     9.35               4326.92          1176.77
41                   Germany    14.66               3436.80           497.05
42                     Ghana     6.32               5052.76           453.02
43                    Greece     5.52               3341.19          1502.79
44                 Guatemala     5.67               3671.83           632.15
45                    Guinea     6.31               3513.10           960.13
46                     Haiti     7.07               3182.94           739.35
47                  Honduras     3.71               4235.64           688.71
48                   Hungary    20.82               5201.95           780.92
49                     India    15.27               4752.96           481.88
50                 Indonesia     3.07               4197.99           713.02
51          

In [23]:
df_2017 = df_2017.reset_index(drop=True)
print(df_2017.shape)
df_2017.tail(1)

(128, 10)


Country name Code    year  Happiness Index  Log GDP  Social support  \
127     Zimbabwe  ZWE  2017.0             3.64     7.85            0.75   

     Healthy life exp  Freedom  Generosity  Pct corruption  
127              55.0     0.75       -0.09            0.75

In [24]:
#merge(self, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes='_x', '_y', copy=True, indicator=False, validate=None) 
df_2017 = pd.merge(df_2017 , GHDx_df, how='left', on='Country name')
df_2017 = df_2017.drop([df_2017.index[29] , df_2017.index[46]])
print(df_2017.shape)
df_2017

(128, 13)


Country name Code    year  Happiness Index  Log GDP  \
0                  Afghanistan  AFG  2017.0             2.66     7.47   
1                      Albania  ALB  2017.0             4.64     9.38   
2                    Argentina  ARG  2017.0             6.04     9.85   
3                      Armenia  ARM  2017.0             4.29     9.08   
4                    Australia  AUS  2017.0             7.26    10.71   
5                      Austria  AUT  2017.0             7.29    10.73   
6                   Azerbaijan  AZE  2017.0             5.15     9.68   
7                      Bahrain  BHR  2017.0             6.23    10.68   
8                   Bangladesh  BGD  2017.0             4.31     8.20   
9                      Belarus  BLR  2017.0             5.55     9.75   
10                     Belgium  BEL  2017.0             6.93    10.66   
11                       Benin  BEN  2017.0             4.85     7.63   
12                     Bolivia  BOL  2017.0             5.65     8.82   
13      Bosnia and Herzegovina  BIH  2017.0             5.09     9.42   
14                    Botswana  BWA  2017.0             3.50     9.69   
15                      Brazil  BRA  2017.0             6.33     9.56   
16                    Bulgaria  BGR  2017.0             5.10     9.83   
17                Burkina Faso  BFA  2017.0             4.65     7.44   
18                    Cambodia  KHM  2017.0             4.59     8.20   
19                    Cameroon  CMR  2017.0             5.07     8.11   
20                      Canada  CAN  2017.0             7.41    10.69   
21                        Chad  TCD  2017.0             4.56     7.47   
22                       Chile  CHL  2017.0             6.32    10.01   
23                       China  CHN  2017.0             5.10     9.63   
24                    Colombia  COL  2017.0             6.16     9.49   
25         Congo (Brazzaville)  COG  2017.0             4.88     8.54   
26                  Costa Rica  CRC  2017.0             7.23     9.64   
27                     Croatia  HRV  2017.0             5.34    10.04   
28                      Cyprus  CYP  2017.0             6.06    10.41   
30              Czech Republic  CZE  2017.0             6.79    10.39   
31                     Denmark  DNK  2017.0             7.59    10.76   
32          Dominican Republic  DOM  2017.0             5.61     9.61   
33                     Ecuador  ECU  2017.0             5.84     9.25   
34                       Egypt  EGY  2017.0             3.93     9.28   
35                 El Salvador  SLV  2017.0             6.34     8.89   
36                     Estonia  EST  2017.0             5.94    10.31   
37                     Finland  FIN  2017.0             7.79    10.63   
38                      France  FRA  2017.0             6.64    10.57   
39                       Gabon  GAB  2017.0             4.78     9.69   
40                     Georgia  GEO  2017.0             4.45     9.18   
41                     Germany  DEU  2017.0             7.07    10.72   
42                       Ghana  GHA  2017.0             5.48     8.31   
43                      Greece  GRC  2017.0             5.15    10.11   
44                   Guatemala  GTM  2017.0             6.33     8.91   
45                      Guinea  GIN  2017.0             4.87     7.70   
47                       Haiti  HTI  2017.0             3.82     7.41   
48                    Honduras  HND  2017.0             6.02     8.40   
49   Hong Kong S.A.R. of China  HKG  2017.0             5.36    10.93   
50                     Hungary  HUN  2017.0             6.07    10.20   
51                       India  IND  2017.0             4.05     8.78   
52                   Indonesia  IDN  2017.0             5.10     9.32   
53                        Iran  IRN  2017.0             4.72     9.86   
54                        Iraq  IRQ  2017.0             4.46     9.67   
55                     Ireland  IRL  2017.0             7.06    11.10   
56 

In [25]:
#check list of rows to be deleted
#outcast_df1 = df_2017[~df_2017['Country name'].isin(GHDx_df['Country name'])].reset_index(drop=True)
#outcast_df1


# Other Files

In [26]:
GDP1_pp = pd.read_csv("../Resources/gdp_df.csv")
GDP_pp = GDP1_pp.drop(list(GDP1_pp)[1:20], axis=1)
GDP_pp = pd.merge(HappyCountries, GDP1_pp, how='left', left_on='Country name', right_on='Country Name',
         left_index=False, right_index=False, sort=True)

df_2017.insert(3,('GDP'),GDP_pp['2017'])

print(df_2017.shape)
df_2017.head(1)

(128, 14)


Country name Code    year     GDP  Happiness Index  Log GDP  Social support  \
0  Afghanistan  AFG  2017.0  2203.0             2.66     7.47            0.49   

   Healthy life exp  Freedom  Generosity  Pct corruption  Suicide  \
0              52.8     0.43       -0.11            0.95     5.25   

   Depression & Anxiety  Substance Abuse  
0               3870.85           687.47

In [27]:
# Doctor
Doctor_file = pd.read_csv("../Outputs/physician to population ratio.csv") 
doctor_df = pd.merge(HappyCountries, Doctor_file, how='left', left_on='Country name',  #fit data to standard dataframe
         right_on='Country Name',left_index=False, right_index=False, sort=True)
df_2017['Physician perpop']= doctor_df['2017']

print(df_2017.shape)
df_2017.head(1)

(128, 15)


Country name Code    year     GDP  Happiness Index  Log GDP  Social support  \
0  Afghanistan  AFG  2017.0  2203.0             2.66     7.47            0.49   

   Healthy life exp  Freedom  Generosity  Pct corruption  Suicide  \
0              52.8     0.43       -0.11            0.95     5.25   

   Depression & Anxiety  Substance Abuse  Physician perpop  
0               3870.85           687.47               NaN

In [28]:
# Average Age
Age_file = pd.read_csv("../Resources/Avg.age2017-final.csv") 

del Age_file['Column1']
Age_file.rename(columns={ Age_file.columns[0]: "Country Name", Age_file.columns[1]:"Average age" }, inplace = True)
Age_file['Country Name'] = Age_file['Country Name'].str.strip()

Age_df = pd.merge(HappyCountries, Age_file, how='left', left_on='Country name',  #fit data to standard dataframe
         right_on='Country Name',left_index=False, right_index=False, sort=True)
df_2017['Average age']= Age_df['Average age']      #add to df_2017 dataframe
print(df_2017.shape)
df_2017.head(1)

(128, 16)


Country name Code    year     GDP  Happiness Index  Log GDP  Social support  \
0  Afghanistan  AFG  2017.0  2203.0             2.66     7.47            0.49   

   Healthy life exp  Freedom  Generosity  Pct corruption  Suicide  \
0              52.8     0.43       -0.11            0.95     5.25   

   Depression & Anxiety  Substance Abuse  Physician perpop  Average age  
0               3870.85           687.47               NaN         18.8

In [29]:
# Sleep
Sleep_file = pd.read_csv("../Resources/Avg_sleep_minutes.csv") 

sleep_df = pd.merge(HappyCountries, Sleep_file, how='left', left_on='Country name',  #fit data to standard dataframe
         right_on='Country',left_index=False, right_index=False, sort=True)
df_2017['Sleep (min)']= sleep_df['Minutes']      #add to df_2017 dataframe
print(df_2017.shape)
df_2017.head(1)

(128, 17)


Country name Code    year     GDP  Happiness Index  Log GDP  Social support  \
0  Afghanistan  AFG  2017.0  2203.0             2.66     7.47            0.49   

   Healthy life exp  Freedom  Generosity  Pct corruption  Suicide  \
0              52.8     0.43       -0.11            0.95     5.25   

   Depression & Anxiety  Substance Abuse  Physician perpop  Average age  \
0               3870.85           687.47               NaN         18.8   

   Sleep (min)  
0          NaN

# TheGlobalEconomy Files

In [30]:
# TheGlobalEconomy data
#Read files
Global1_file = pd.read_csv("../Resources/TheGlobalEconomy1.csv")  
Global2_file = pd.read_csv("../Resources/TheGlobalEconomy2.csv") 

In [31]:
Global1_file.rename(columns={"Gini income inequality index":"Gini index","Poverty percent of population":"Pct Poverty",
                            "Public spending on education percent of public spending":"Pct spent ed",
                             "International tourism revenue":"Rev tourism"},inplace=True)
Global2_file.rename(columns={"Fragile state index 0 (low) - 120 (high)":"Fragile State index",
                             "Security threats index 0 (low) - 10 (high)":"Security index",
                             "Human flight and brain drain index 0 (low) - 10 (high)":"Brain drain"},inplace=True)

print(Global1_file.shape)
Global2_file.head(1)

(2288, 14)


Country Code  Year  Labor force million people  Unemployment rate  \
0  Afghanistan  AFG  2007                        6.75              11.05   

   Unemployment rate for females  Unemployment rate for males  \
0                          14.72                        10.38   

   Youth unemployment ages 15-24  Labor force participation rate  \
0                          17.25                            48.0   

   Labor force percent female  ...  Percent urban population  \
0                        15.4  ...                     23.11   

   Population density people per square km  Life expectancy in years  \
0                                     42.0                     59.38   

   Ratio of female to male pupils in primary school  \
0                                              0.61   

   Tertiary school enrollment percent of all eligible children  \
0                                                NaN             

   Percent of world GDP  Fragile State index  Security index  \
0                  0.02                102.3             9.0   

   Uneven economic development index 0 (low) - 10 (high)  Brain drain  
0                                                8.0              7.0  

[1 rows x 23 columns]

In [32]:
Global1_df = Global1_file[Global1_file["Year"]==2017]
Global1_df = Global1_df.filter(['Code','Gini index','Pct Poverty','Pct spent ed','Literacy rate','Tourist arrivals','Rev tourism'], axis=1).reset_index(drop=True)  
Global_df = pd.merge(HappyCountries, Global1_df, how='left', on=('Code')).reset_index(drop=True)

Global2_df = Global2_file[Global2_file["Year"]==2017]
Global2_df = Global2_df.filter(['Code','Fragile State index','Security index','Brain drain','Unemployment rate'], axis=1).reset_index(drop=True)  
Global_df = pd.merge(Global_df, Global2_df, how='left', on=('Code')).reset_index(drop=True)

print(Global_df.shape)
Global_df.head(1)

(128, 12)


Country name Code  Gini index  Pct Poverty  Pct spent ed  Literacy rate  \
0  Afghanistan  AFG         NaN          NaN         15.66            NaN   

   Tourist arrivals  Rev tourism  Fragile State index  Security index  \
0               NaN         16.0                107.3            10.0   

   Brain drain  Unemployment rate  
0          8.2              11.18

In [33]:
df_2017 = pd.merge(df_2017, Global_df, how='left', on=('Country name','Code')).reset_index(drop=True)
print(df_2017.shape)
df_2017.head(1)

(128, 27)


Country name Code    year     GDP  Happiness Index  Log GDP  Social support  \
0  Afghanistan  AFG  2017.0  2203.0             2.66     7.47            0.49   

   Healthy life exp  Freedom  Generosity  ...  Gini index  Pct Poverty  \
0              52.8     0.43       -0.11  ...         NaN          NaN   

   Pct spent ed  Literacy rate  Tourist arrivals  Rev tourism  \
0         15.66            NaN               NaN         16.0   

   Fragile State index  Security index  Brain drain  Unemployment rate  
0                107.3            10.0          8.2              11.18  

[1 rows x 27 columns]

# Normalisation

In [34]:
min_value = df_2017.min()
max_value = df_2017.max()

norml_df = pd.DataFrame({'Min Value':min_value,'Max Value':max_value})
norml_df.drop(norml_df.index[:3], inplace=True)
norml_df['range'] = norml_df['Max Value'] - norml_df['Min Value']

norml_df.to_csv("../Outputs/Normalization value 2017.csv", index=True)
norml_df

Min Value Max Value    range
GDP                        847    112823   111976
Happiness Index           2.66      7.79     5.13
Log GDP                   6.83     11.44     4.61
Social support            0.44      0.96     0.52
Healthy life exp          47.7      76.5     28.8
Freedom                   0.43      0.99     0.56
Generosity               -0.29      0.48     0.77
Pct corruption            0.16      0.95     0.79
Suicide                   2.86     36.27    33.41
Depression & Anxiety   1843.64   7022.08  5178.44
Substance Abuse         296.64   2282.46  1985.82
Physician perpop        0.0434    6.1297   6.0863
Average age               15.4      47.3     31.9
Sleep (min)                377       458       81
Gini index                25.4      53.3     27.9
Pct Poverty                2.4        70     67.6
Pct spent ed              6.95     30.15     23.2
Literacy rate             48.3     99.79    51.49
Tourist arrivals            11     86758    86747
Rev tourism                 16    251544   251528
Fragile State index       18.7     111.1     92.4
Security index               1        10        9
Brain drain                1.3       8.8      7.5
Unemployment rate         0.49     27.07    26.58

In [35]:
col_2017 = df_2017.keys()
print(col_2017)

Index(['Country name', 'Code', 'year', 'GDP', 'Happiness Index', 'Log GDP',
       'Social support', 'Healthy life exp', 'Freedom', 'Generosity',
       'Pct corruption', 'Suicide', 'Depression & Anxiety', 'Substance Abuse',
       'Physician perpop', 'Average age', 'Sleep (min)', 'Gini index',
       'Pct Poverty', 'Pct spent ed', 'Literacy rate', 'Tourist arrivals',
       'Rev tourism', 'Fragile State index', 'Security index', 'Brain drain',
       'Unemployment rate'],
      dtype='object')


In [36]:
#NORMALIZATION TABLE 2017
norm_2017_df = country_code.copy()   
for i in range(3,len(col_2017)):
    norm_2017_df[col_2017 [i]]= (df_2017[col_2017 [i]] - norml_df.loc[col_2017 [i],'Min Value']) / norml_df.loc[col_2017 [i],'range']

norm_2017_df['Blue Index'] = norm_2017_df[['Suicide','Depression & Anxiety','Substance Abuse']].mean(axis=1)
norm_2017_df.to_csv("../Outputs/Normalized Table 2017_df.csv", index=False)
norm_2017_df.round(2)

Country name Code   GDP  Happiness Index  Log GDP  \
0                  Afghanistan  AFG  0.01             0.00     0.14   
1                      Albania  ALB  0.11             0.39     0.55   
2                    Argentina  ARG  0.20             0.66     0.66   
3                      Armenia  ARM  0.10             0.32     0.49   
4                    Australia  AUS  0.43             0.90     0.84   
5                      Austria  AUT  0.48             0.90     0.85   
6                   Azerbaijan  AZE  0.12             0.49     0.62   
7                      Bahrain  BHR  0.42             0.70     0.84   
8                   Bangladesh  BGD  0.03             0.32     0.30   
9                      Belarus  BLR  0.16             0.56     0.63   
10                     Belgium  BEL  0.45             0.83     0.83   
11                       Benin  BEN  0.02             0.43     0.17   
12                     Bolivia  BOL  0.07             0.58     0.43   
13      Bosnia and Herzegovina  BIH  0.12             0.47     0.56   
14                    Botswana  BWA  0.15             0.16     0.62   
15                      Brazil  BRA  0.12             0.72     0.59   
16                    Bulgaria  BGR  0.18             0.48     0.65   
17                Burkina Faso  BFA  0.01             0.39     0.13   
18                    Cambodia  KHM  0.03             0.38     0.30   
19                    Cameroon  CMR  0.02             0.47     0.28   
20                      Canada  CAN  0.43             0.93     0.84   
21                        Chad  TCD  0.01             0.37     0.14   
22                       Chile  CHL  0.20             0.71     0.69   
23                       China  CHN  0.12             0.48     0.61   
24                    Colombia  COL  0.12             0.68     0.58   
25         Congo (Brazzaville)  COG   NaN             0.43     0.37   
26                  Costa Rica  CRC  0.16             0.89     0.61   
27                     Croatia  HRV  0.23             0.52     0.70   
28                      Cyprus  CYP  0.29             0.66     0.78   
29              Czech Republic  CZE  0.48             0.81     0.77   
30                     Denmark  DNK  0.14             0.96     0.85   
31          Dominican Republic  DOM  0.10             0.58     0.60   
32                     Ecuador  ECU  0.09             0.62     0.52   
33                       Egypt  EGY  0.07             0.25     0.53   
34                 El Salvador  SLV  0.29             0.72     0.45   
35                     Estonia  EST  0.42             0.64     0.75   
36                     Finland  FIN  0.39             1.00     0.82   
37                      France  FRA  0.13             0.78     0.81   
38                       Gabon  GAB  0.11             0.41     0.62   
39                     Georgia  GEO  0.47             0.35     0.51   
40                     Germany  DEU  0.04             0.86     0.84   
41                       Ghana  GHA  0.25             0.55     0.32   
42                      Greece  GRC  0.07             0.49     0.71   
43                   Guatemala  GTM  0.01             0.72     0.45   
44                      Guinea  GIN  0.01             0.43     0.19   
45                       Haiti  HTI   NaN             0.23     0.13   
46                    Honduras  HND  0.26             0.65     0.34   
47   Hong Kong S.A.R. of China  HKG  0.05             0.53     0.89   
48                     Hungary  HUN  0.09             0.66     0.73   
49                       India  IND  0.12             0.27     0.42   
50                   Indonesia  IDN  0.09             0.48     0.54   
51                        Iran  IRN  0.69             0.40     0.66   
52                        Iraq  IRQ  0.34             0.35     0.62   
53                     Ireland  IRL  0.37             0.86     0.93   
54                      Israel  ISR  0.36             0.91     0.78   
55                       Italy  ITA  0.08 

In [37]:
norm_2017_df.columns

Index(['Country name', 'Code', 'GDP', 'Happiness Index', 'Log GDP',
       'Social support', 'Healthy life exp', 'Freedom', 'Generosity',
       'Pct corruption', 'Suicide', 'Depression & Anxiety', 'Substance Abuse',
       'Physician perpop', 'Average age', 'Sleep (min)', 'Gini index',
       'Pct Poverty', 'Pct spent ed', 'Literacy rate', 'Tourist arrivals',
       'Rev tourism', 'Fragile State index', 'Security index', 'Brain drain',
       'Unemployment rate', 'Blue Index'],
      dtype='object')